## Курсова робота на тему "Розрахунок необхідної кількості робітників та оптимального розподілу їх між верстатами для максимізації прибутку від роботи ділянки по складанню вузлів"
### Виконав студент гр. ІС-72 Фрідріхсон Михайло

In [1]:
# count = 0
# count_half = 0

In [2]:
from collections import defaultdict
from copy import copy
from enum import Enum
from IPython.display import display
import numpy as np
import pandas as pd
import uuid

In [3]:
class Distribution(Enum):
    EXP = 0
    UNIFORM = 1
    NORMAL = 2
    POISSON = 3

In [4]:
class Position:
    def __init__(self, num_of_markers=0, description=None):
        self.markers = [Marker() for _ in range(num_of_markers)]
        self.description = description

        self.in_arcs = []
        self.out_arcs = []

    def add_markers(self, markers):
        self.markers.extend(markers)

    def remove_markers(self, num_of_markers):
        markers_to_remove = self.markers[:num_of_markers]
        del self.markers[:num_of_markers]

        return markers_to_remove

    def __repr__(self):
        return self.description

In [5]:
class Transition:
    def __init__(
        self,
        delay=0,
        delay_distribution=None,
        delay_distribution_params=None,
        priority=1,
        probability=None,
        description=None
    ):
        self.delay = delay
        self.delay_distribution = delay_distribution
        self.delay_distribution_params = delay_distribution_params
        self.priority = priority
        self.probability = probability
        self.description = description

        self.in_arcs = []
        self.out_arcs = []
        
        # {time_of_release: Array<Marker>}
        self.marker_release_timestamps = {}

    def get_random_delay_period(self):
        if self.delay_distribution is Distribution.EXP:
            result = np.random.exponential(**self.delay_distribution_params)
        elif self.delay_distribution is Distribution.UNIFORM:
            result = np.random.uniform(**self.delay_distribution_params)
        elif self.delay_distribution is Distribution.NORMAL:
            result = np.random.normal(**self.delay_distribution_params)
        elif self.delay_distribution is Distribution.POISSON:
            result = np.random.poisson(**self.delay_distribution_params)

        return round(result)

    def check_if_can_perform_transition(self):
        return all(arc.check_start_markers() for arc in self.in_arcs)

    def check_delayed_markers(self, current_time):
#         print(self.marker_release_timestamps.keys())
        if current_time in self.marker_release_timestamps.keys():
            print(self.marker_release_timestamps)
            marker_sample = self.marker_release_timestamps[current_time][0]
            del self.marker_release_timestamps[current_time]

            for arc in self.out_arcs:
                arc.move_to(marker_sample)

    def perform_transition(self, current_time):
        for arc in self.in_arcs:
            arc.move_from(current_time)

    def __repr__(self):
        return self.description

In [6]:
class BendingTransition(Transition):
    """
    
    """
    def modify_marker(self, marker, current_time):
        marker.state["pair_id"] = str(uuid.uuid4())
        marker.state["processing_start_time"] = current_time

        return marker

    def check_delayed_markers(self, current_time):
        if current_time in self.marker_release_timestamps.keys():
            marker_sample = self.marker_release_timestamps[current_time][0]
            del self.marker_release_timestamps[current_time]

            for arc in self.out_arcs:
                arc.move_to(self.modify_marker(marker_sample, current_time))

In [7]:
class Arc:
    def __init__(self, start, end, multiplier=1, informational=False):
        self.start = start
        self.end = end
        self.multiplier = multiplier
        self.informational = informational

        self.update_arc_ends()

    def update_arc_ends(self):
        self.start.out_arcs.append(self)
        self.end.in_arcs.append(self)

    def check_start_markers(self):
        return len(self.start.markers) >= self.multiplier

    def move_from(self, current_time):
        # self.start - Position
        # self.end - Transition
        
        markers = []
        if not self.informational:
            markers = self.start.remove_markers(self.multiplier)

        delay = self.end.delay if self.end.delay_distribution is None else self.end.get_random_delay_period()

        release_timestamp = current_time + delay
        if release_timestamp in self.end.marker_release_timestamps:
            self.end.marker_release_timestamps[release_timestamp].extend(markers)
        else:
            self.end.marker_release_timestamps[release_timestamp] = markers

    def move_to(self, marker_sample):
        # self.start - Transition
        # self.end - Position
        markers = [copy(marker_sample) for _ in range(round(self.multiplier))]

        self.end.add_markers(markers)

    def __repr__(self):
        return f"{self.start} -> {self.end}"

In [8]:
class RevenueArc(Arc):
    def move_to(self, marker_sample):
        revenue = self.multiplier
        
        print(round(revenue))
        markers = [copy(marker_sample) for _ in range(round(revenue))]

        self.end.add_markers(markers)

In [9]:
class Marker:
    def __init__(self, initial_state={}):
        self.state = initial_state

    def __repr__(self):
        return "•"

In [10]:
class Model:
    def __init__(self, transitions, positions, modeling_period, should_print_intermediate_results=False):
        self.transitions = transitions
        self.positions = positions
        self.modeling_period = modeling_period
        self.should_print_intermediate_results = should_print_intermediate_results
        self.time = 0

        # Stats
        self.position_marker_stats = {position: [] for position in positions}

    def run(self):
        if self.should_print_intermediate_results:
            self.print_intermediate_results()

        while self.time < self.modeling_period:
            can_perform_transitions = any(
                transition.check_if_can_perform_transition() or len(transition.marker_release_timestamps) > 0
                for transition in self.transitions
            )

            if not can_perform_transitions:
                break

            transitions_without_conflicts = self.get_transitions_with_resolved_conflicts()

            for transition in transitions_without_conflicts:
                if transition.check_if_can_perform_transition():
                    transition.perform_transition(self.time)

            for transition in self.transitions:
                transition.check_delayed_markers(self.time)

            self.calc_stats()

            self.time += 1

            if self.should_print_intermediate_results:
                self.print_intermediate_results()

        return self.print_results()

    def get_transitions_with_resolved_conflicts(self):
        conflicting_transitions = []
        resulting_transitions = []

        for position in self.positions:
            non_informational_arcs = filter(lambda arc: not arc.informational, position.out_arcs)

            valid_transitions = [*filter(
                lambda transition: transition.check_if_can_perform_transition(),
                map(lambda arc: arc.end, non_informational_arcs)
            )]

            if len(valid_transitions) > 1:
                conflicting_transitions.append(valid_transitions)

        for conflict in conflicting_transitions:
            is_probabilistic_conflict = conflict[0].probability is not None

            if is_probabilistic_conflict:
                probabilities = [*map(lambda transition: transition.probability, conflict)]

                gap = 1 - sum(probabilities)
                if gap != 0:

                    probabilities = [p + gap / len(probabilities) for p in probabilities]

                resulting_transitions.append(np.random.choice(conflict, p=probabilities))
            else:
                conflict.sort(key=lambda transition: transition.priority, reverse=True)
                resulting_transitions.append(conflict[0])


        for transition in self.transitions:
            was_transition_in_conflict = any([transition in conflict for conflict in conflicting_transitions])

            if not was_transition_in_conflict:
                resulting_transitions.append(transition)

        return resulting_transitions

    def calc_stats(self):
        for position in self.positions:
            self.position_marker_stats[position].append(len(position.markers))

    def get_result_stats(self):
        result_stats = {position: {} for position in self.positions}

        for position, stats in self.position_marker_stats.items():
            result_stats[position]["avg_markers"] = sum(stats) / len(stats)
            result_stats[position]["max_markers"] = max(stats)
            result_stats[position]["min_markers"] = min(stats)
            result_stats[position]["result_markers"] = stats[len(stats) - 1]

        return result_stats

    def print_results(self):
        result_stats = self.get_result_stats()
        formatted_records = []

        for position, stats in result_stats.items():
            record = stats
            record["description"] = position.description

            formatted_records.append(record)

        print(f"Modeling time spent: {self.time}")
        display(pd.DataFrame(
            formatted_records,
            columns=[
                "description",
                "avg_markers",
                "max_markers",
                "min_markers",
                "result_markers"
            ]
        ))

        return result_stats

    def print_intermediate_results(self):
        print(f"Timestamp: {self.time}")
        print("--- Positions ---")
        for position in self.positions:
            print(f"{position}: {len(position.markers)}")
        print("--- Transitions ---")
        for t in self.transitions:
            print(t, t.marker_release_timestamps)
        print()

### Моделювання задачі

In [13]:
T1 = 20
T1_RANGE = 8

T2_basic = 400
T2 = 400
T2_RANGE = 50

T3 = 5

T4 = 100
T4_RANGE = 25

s1 = 60
s2 = 30
s3 =0.04

R = 18
R_RANGE = 10

k = np.abs(T2_basic - T2)

modeling_period = 10000

# Positions
generator = Position(num_of_markers=1, description="Надходження")
queue = Position(num_of_markers=0, description="Накопичувач (черга)")

basic_channel_is_available = Position(num_of_markers=1, description="Основний канал вільний")
reserv_channel_is_available = Position(num_of_markers=1, description="Резервний канал вільний")

buffer = Position(num_of_markers=0, description="Буфер")
innterupts_cnt = Position(num_of_markers=0, description="Кількість переривань")

basic_result_revenue = Position(num_of_markers=0, description="Прибуток від передачі основним каналом")
basic_result = Position(num_of_markers=0, description="Передано основним каналом")

general_result_revenue = Position(num_of_markers=0, description="Загальний прибуток від передачі")
general_result = Position(num_of_markers=0, description="Загальна кількість переданих повідомлень")

reserv_result_revenue = Position(num_of_markers=0, description="Прибуток від передачі резервним каналом")
reserv_result = Position(num_of_markers=0, description="Передано резервним каналом")

accessibility_marker = Position(num_of_markers=1, description="Маркер доступу до основного каналу")

total_failurs = Position(num_of_markers=0, description="Загальна кількість збоїв")

p2 = Position(num_of_markers=1, description="P2")
fixed_failure = Position(num_of_markers=0, description="Збій усунено")
p1 = Position(num_of_markers=1, description="P1")

was_failure = Position(num_of_markers=0, description="Відбувся збій")
need_to_run_reserv_channel = Position(num_of_markers=0, description="Потрібно запустити резервний канал")
reserv_channel_has_been_started = Position(num_of_markers=0, description="Резервний канал запущено")

# Transitions
message_arrived_T = BendingTransition(
    delay_distribution=Distribution.UNIFORM,
    delay_distribution_params={"low": R - R_RANGE, "high": R + R_RANGE},
    description="Перехід до накопичувача"
)

basic_channel_T = Transition(
    delay_distribution=Distribution.UNIFORM,
    delay_distribution_params={"low": T1 - T1_RANGE, "high": T1 + T1_RANGE},
    description="Основний канал"
)

sending_finished_T = Transition(
    delay=0,
    description="Завершення надсилання"
)

innterupting_T = Transition(
    delay=0,
    priority=2,
    description="Переривання"
)

reserv_chanel_T = Transition(
    delay_distribution=Distribution.UNIFORM,
    delay_distribution_params={"low": T1 - T1_RANGE, "high": T1 + T1_RANGE},
    priority=2,
    description="Резервний канал"
)

recording_failure_T = Transition(
    delay=0,
    description="Запис збою"
)

fixing_failure_T = Transition(
    delay_distribution=Distribution.UNIFORM,
    delay_distribution_params={"low": T4 - T4_RANGE, "high": T4 + T4_RANGE},
    description="Усунення збою"
)

returning_marker_T = Transition(
    delay=0,
    description="Повернення маркеру"
)

failure_generation_T = Transition(
    delay_distribution=Distribution.UNIFORM,
    delay_distribution_params={"low": T2 - T2_RANGE, "high": T2 + T2_RANGE},
    description="Генерація збою"
)

starting_reserv_channel_T = Transition(
    delay=T3,
    description="Запуск резервного каналу"
)


# Arcs
a1 = Arc(start=generator, end=message_arrived_T)
a_1 = Arc(start=message_arrived_T, end=generator)

a2 = Arc(start=message_arrived_T, end=queue)

a3 = Arc(start=queue, end=basic_channel_T)
a4 = Arc(start=innterupting_T, end=queue)
a5 = Arc(start=queue, end=reserv_chanel_T)

a6 = Arc(start=accessibility_marker, end=basic_channel_T, informational=True)
a7 = Arc(start=basic_channel_T, end=buffer)
a21 = Arc(start=basic_channel_T, end=basic_channel_is_available)
a22 = Arc(start=basic_channel_is_available, end=basic_channel_T)

a8 = Arc(start=buffer, end=sending_finished_T)
a9 = Arc(start=buffer, end=innterupting_T)

a10 = RevenueArc(start=sending_finished_T, end=basic_result_revenue, multiplier=s1-k*s3)
a11 = Arc(start=sending_finished_T, end=basic_result)
a12 = RevenueArc(start=sending_finished_T, end=general_result_revenue, multiplier=s1-k*s3)
a13 = Arc(start=sending_finished_T, end=general_result)

a14 = Arc(start=innterupting_T, end=innterupts_cnt)
a24 = Arc(start=was_failure, end=innterupting_T, informational=True)

a15 = RevenueArc(start=reserv_chanel_T, end=general_result_revenue, multiplier=s2-k*s3)
a16 = Arc(start=reserv_chanel_T, end=general_result)
a17 = RevenueArc(start=reserv_chanel_T, end=reserv_result_revenue, multiplier=s2-k*s3)
a18 = Arc(start=reserv_chanel_T, end=reserv_result)

a19 = Arc(start=reserv_chanel_T, end=reserv_channel_is_available)
a20 = Arc(start=reserv_channel_is_available, end=reserv_chanel_T)

a23 = Arc(start=reserv_channel_has_been_started, end=reserv_chanel_T, informational=True)
a24 = Arc(start=reserv_channel_has_been_started, end=returning_marker_T)
a25 = Arc(start=starting_reserv_channel_T, end=reserv_channel_has_been_started)

a26 = Arc(start=need_to_run_reserv_channel, end=starting_reserv_channel_T)

a27 = Arc(start=failure_generation_T, end=need_to_run_reserv_channel)
a28 = Arc(start=failure_generation_T, end=was_failure)
a29 = Arc(start=was_failure, end=fixing_failure_T, informational=True)
a30 = Arc(start=was_failure, end=recording_failure_T, informational=True)
a33 = Arc(start=was_failure, end=returning_marker_T)

a31 = Arc(start=p2, end=fixing_failure_T)
a32 = Arc(start=returning_marker_T, end=p2)

a34 = Arc(start=fixing_failure_T, end=fixed_failure)
a35 = Arc(start=fixed_failure, end=returning_marker_T)

a36 = Arc(start=returning_marker_T, end=accessibility_marker)
a37 = Arc(start=returning_marker_T, end=p1)

a38 = Arc(start=p1, end=failure_generation_T)

a39 = Arc(start=accessibility_marker, end=recording_failure_T)
a40 = Arc(start=recording_failure_T, end=total_failurs)

net = Model(
    positions=[
        generator,
        queue,

        basic_channel_is_available,
        reserv_channel_is_available,

        buffer,
        innterupts_cnt,

        basic_result_revenue,
        basic_result,

        general_result_revenue,
        general_result,

        reserv_result_revenue,
        reserv_result,

        accessibility_marker,

        total_failurs,

        p2,
        fixed_failure,
        p1,

        was_failure,
        need_to_run_reserv_channel,
        reserv_channel_has_been_started
    ],
    transitions=[
        message_arrived_T,

        basic_channel_T,

        sending_finished_T,

        innterupting_T,

        reserv_chanel_T,

        recording_failure_T,

        fixing_failure_T,

        returning_marker_T,

        failure_generation_T,

        starting_reserv_channel_T

    ],
    modeling_period=modeling_period,
    should_print_intermediate_results=True
)
result = net.run()

Timestamp: 0
--- Positions ---
Надходження: 1
Накопичувач (черга): 0
Основний канал вільний: 1
Резервний канал вільний: 1
Буфер: 0
Кількість переривань: 0
Прибуток від передачі основним каналом: 0
Передано основним каналом: 0
Загальний прибуток від передачі: 0
Загальна кількість переданих повідомлень: 0
Прибуток від передачі резервним каналом: 0
Передано резервним каналом: 0
Маркер доступу до основного каналу: 1
Загальна кількість збоїв: 0
P2: 1
Збій усунено: 0
P1: 1
Відбувся збій: 0
Потрібно запустити резервний канал: 0
Резервний канал запущено: 0
--- Transitions ---
Перехід до накопичувача {}
Основний канал {}
Завершення надсилання {}
Переривання {}
Резервний канал {}
Запис збою {}
Усунення збою {}
Повернення маркеру {}
Генерація збою {}
Запуск резервного каналу {}

Timestamp: 1
--- Positions ---
Надходження: 0
Накопичувач (черга): 0
Основний канал вільний: 1
Резервний канал вільний: 1
Буфер: 0
Кількість переривань: 0
Прибуток від передачі основним каналом: 0
Передано основним канало

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [49]:
# LAMBDA_1 = 1/40
# LAMBDA_2 = 1/40

# T1 = 55
# T1_RANGE = 12

# T2 = 40
# T2_RANGE = 10

# T3 = 25
# T3_RANGE = 6

# T4 = 48
# T4_RANGE = 12

# p1 = 0.7
# p2 = 0.55

# s1 = 600
# s2 = 90

# T = 40

# modeling_period = 480

# # Positions
# type_1_factory = Position(num_of_markers=1, description="Потік вузлів типу 1")
# type_2_factory = Position(num_of_markers=1, description="Потік вузлів типу 2")

# type_1_available = Position(num_of_markers=0, description="Вузол типу 1 доступний")
# type_2_available = Position(num_of_markers=0, description="Вузол типу 2 доступний")

# type_1_was_bent = Position(num_of_markers=0, description="Вузол типу 1 пройшов операцію пригінки")
# type_2_was_bent = Position(num_of_markers=0, description="Вузол типу 2 пройшов операцію пригінки")

# type_1_available_for_assembling = Position(num_of_markers=0, description="Вузол типу 1 доступний для складання")
# type_2_available_for_assembling = Position(num_of_markers=0, description="Вузол типу 2 доступний для складання")

# assembling_finished = Position(num_of_markers=0, description="Операцію складання проведено")

# total_revenue = Position(num_of_markers=0, description="Загальний прибуток")

# # Transitions
# type_1_arrived = Transition(
#     delay_distribution=Distribution.POISSON,
#     delay_distribution_params={"lam":LAMBDA_1},
#     description="Надходження вузла типу 1"
# )
# type_2_arrived = Transition(
#     delay_distribution=Distribution.POISSON,
#     delay_distribution_params={"lam":LAMBDA_2},
#     description="Надходження вузла типу 2"
# )

# bending = BendingTransition(
#     delay_distribution=Distribution.UNIFORM,
#     delay_distribution_params={"low": T1 - T1_RANGE, "high": T1 + T1_RANGE},
#     description="Проведення операції пригінки"
# )

# type_1_finalizing = Transition(
#     probability=p1,
#     delay_distribution=Distribution.UNIFORM,
#     delay_distribution_params={"low": T2 - T2_RANGE, "high": T2 + T2_RANGE},
#     description="Проведення операції довдення вузла типу 1"
# )
# type_1_to_ready = Transition(probability=1-p1, description="Переведення вузла типу 1 до складання")
# type_2_finalizing = Transition(
#     probability=p2, 
#     delay_distribution=Distribution.UNIFORM,
#     delay_distribution_params={"low": T3 - T3_RANGE, "high": T3 + T3_RANGE},
#     description="Проведення операції довдення вузла типу 2"
# )
# type_2_to_ready = Transition(probability=1-p2, description="Переведення вузла типу 2 до складання")

# assembling = AssemblingTransition(
#     delay_distribution=Distribution.UNIFORM,
#     delay_distribution_params={"low": T4 - T4_RANGE, "high": T4 + T4_RANGE}, 
#     description="Проведення операції складання"
# )

# get_revenue = Transition(description="Отримано повний або частковий прибуток")

# # Arcs
# a1 = Arc(start=type_1_factory, end=type_1_arrived)
# a_1 = Arc(start=type_1_arrived, end=type_1_factory)
# a2 = Arc(start=type_2_arrived, end=type_2_factory)
# a_2 = Arc(start=type_2_factory, end=type_2_arrived)

# a3 = Arc(start=type_1_arrived, end=type_1_available)
# a4 = Arc(start=type_2_arrived, end=type_2_available)

# a5 = Arc(start=type_1_available, end=bending)
# a6 = Arc(start=type_2_available, end=bending)

# a8 = Arc(start=bending, end=type_1_was_bent)
# a9 = Arc(start=bending, end=type_2_was_bent)

# a10 = Arc(start=type_1_was_bent, end=type_1_finalizing)
# a11 = Arc(start=type_1_was_bent, end=type_1_to_ready)
# a12 = Arc(start=type_1_finalizing, end=type_1_available_for_assembling)
# a13 = Arc(start=type_1_to_ready, end=type_1_available_for_assembling)
# a14 = Arc(start=type_1_available_for_assembling, end=assembling)

# a15 = Arc(start=type_2_was_bent, end=type_2_finalizing)
# a16 = Arc(start=type_2_was_bent, end=type_2_to_ready)
# a17 = Arc(start=type_2_finalizing, end=type_2_available_for_assembling)
# a18 = Arc(start=type_2_to_ready, end=type_2_available_for_assembling)
# a19 = Arc(start=type_2_available_for_assembling, end=assembling)

# a21 = Arc(start=assembling, end=assembling_finished)
# a22 = Arc(start=assembling_finished, end=get_revenue)
# a23 = RevenueArc(start=get_revenue, end=total_revenue, multiplier=s1)

# net = Model(
#     positions=[
#         type_1_factory,
#         type_2_factory,

#         type_1_available,
#         type_2_available,

#         type_1_was_bent,
#         type_2_was_bent,

#         type_1_available_for_assembling,
#         type_2_available_for_assembling,

#         assembling_finished,

#         total_revenue
#     ],
#     transitions=[
#         type_1_arrived,
#         type_2_arrived,

#         bending,

#         type_1_finalizing,
#         type_1_to_ready,
#         type_2_finalizing,
#         type_2_to_ready,

#         assembling,

#         get_revenue
#     ],
#     modeling_period=modeling_period,
#     should_print_intermediate_results=False
# )
# result = net.run()
# print(count, count_half)

In [26]:
assembling.marker_release_timestamps

{}